<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Drought Results

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook loads drought results.


In [1]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils, prepare

In [2]:
setup.plotting(remote=False)

In [3]:
%load_ext autoreload
%autoreload 2

Load the results


In [4]:
pf = pd.read_parquet(
    "/Users/freeman/github/carbonplan/forests/scripts/data/drought.parquet"
)

Filter out bad values


In [5]:
badinds = (pf["historical"] > 1) | (np.isnan(pf["historical"]))
for key in pf.columns:
    if key not in ["lat", "lon", "type_code", "r2"]:
        badinds = badinds | ((pf[key] > 1) | (np.isnan(pf[key])))
pf = pf[~badinds]

Make a map of the historical prediction (from the period used for fitting)


In [6]:
pf.columns

Index(['lat', 'lon', 'type_code', 'historical', 'BCC-CSM2-MR_ssp245_2020',
       'BCC-CSM2-MR_ssp370_2020', 'BCC-CSM2-MR_ssp585_2020',
       'BCC-CSM2-MR_ssp245_2040', 'BCC-CSM2-MR_ssp370_2040',
       'BCC-CSM2-MR_ssp585_2040', 'BCC-CSM2-MR_ssp245_2060',
       'BCC-CSM2-MR_ssp370_2060', 'BCC-CSM2-MR_ssp585_2060',
       'BCC-CSM2-MR_ssp245_2080', 'BCC-CSM2-MR_ssp370_2080',
       'BCC-CSM2-MR_ssp585_2080', 'BCC-CSM2-MR_ssp245_2100',
       'BCC-CSM2-MR_ssp370_2100', 'BCC-CSM2-MR_ssp585_2100', 'r2'],
      dtype='object')

In [7]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["historical"],
    cmap="blues",
    clim=(0, 0.3),
    size=2,
)

alt.LayerChart(...)

Make a time series of spatially averaged projections


In [8]:
years = np.arange(2020, 2120, 20)
y1 = [pf["BCC-CSM2-MR_ssp245_" + str(year)].mean() for year in years]
y2 = [pf["BCC-CSM2-MR_ssp370_" + str(year)].mean() for year in years]
y3 = [pf["BCC-CSM2-MR_ssp585_" + str(year)].mean() for year in years]
opts = {
    "color": "rgb(70,112,164)",
    "ylabel": "fractional mortality",
    "xlabel": "year",
}
(
    plot.line(x=years, y=y1, opacity=0.5, ylim=[0.06, 0.07], **opts)
    + plot.line(x=years, y=y2, opacity=0.75, **opts)
    + plot.line(x=years, y=y3, opacity=1, **opts)
)

alt.LayerChart(...)